In [ ]:
from herbie import Herbie
from herbie.toolbox import EasyMap, pc

from herbie import paint

import matplotlib.pyplot as plt

In [ ]:
# Just open the entire GRIB2 files (opened in multiple DataSets)
hrrr = Herbie("2024-06-11 00:00", model="hrrr").xarray(verbose=True)
rtma = Herbie("2024-06-11 00:00", model="rtma").xarray(verbose=True)

In [ ]:
hrrr[3]

In [ ]:
rtma[3]

In [19]:
print("HRRR:", hrrr[3].herbie.crs)
print("RTMA:", rtma[3].herbie.crs)

HRRR: +proj=lcc +ellps=sphere +a=6371229.0 +b=6371229.0 +lon_0=262.5 +lat_0=38.5 +x_0=0.0 +y_0=0.0 +lat_1=38.5 +lat_2=38.5 +no_defs +type=crs
RTMA: +proj=lcc +ellps=sphere +a=6371200.0 +b=6371200.0 +lon_0=265.0 +lat_0=25.0 +x_0=0.0 +y_0=0.0 +lat_1=25.0 +lat_2=25.0 +no_defs +type=crs


In [ ]:
crs = rtma[3].herbie.crs

ax = EasyMap("50m", crs=crs).STATES().ax

art = ax.pcolormesh(
    rtma[3].longitude,
    rtma[3].latitude,
    (rtma[3].t2m - hrrr[3].t2m),
    transform=pc,
    cmap="herbie.white",  # this is a Custom herbie colormap :)
    vmax=3,
    vmin=-3,
)

ax.set_title("RTMA minus URMA")

art = plt.colorbar(
    art,
    ax=ax,
    label="2 m Temperature Difference (Celsius)",
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
    extend="both",
)